In [1]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 13 
num_classes = 7

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [2]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [3]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017Kharif_NDVI1.csv', delimiter=',')

In [4]:
data = my_data[1:,2:]
print(data.shape)

(546, 13)


In [5]:
print(data.max())

0.855952701


In [6]:
data = data/data.max()
print(data.max())

1.0


In [7]:
import numpy as np
data = np.array(data)
print(data.shape)

(546, 13)


In [8]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data1 = pandas.read_csv('./DataForBiplab/FieldDataKharif2017.csv', names=colnames)

In [9]:
ctype = data1.CropType.tolist()
print(ctype)

['CropType', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcan

In [10]:
p = np.array(ctype)
print(p.shape)

(547,)


In [11]:
lbl1 = ctype
label = []
for i in range(1,547):
    if(lbl1[i] == "Paddy"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "PearlMillet"):
        label += [2]
    if(lbl1[i] == "PigeonPea"):
        label += [3]
    if(lbl1[i] == "Sorghum"):
        label += [4]
    if(lbl1[i] == "Forest/NonCropland"):
        label += [5]
    if(lbl1[i] == "Habitation"):
        label += [6]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [12]:
label = np.array(label)
print(label.shape)

(546,)


In [13]:
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data[i])
        l1.append([1,0,0,0,0,0,0])
    if(label[i]==1):
        x2.append(data[i])
        l2.append([0,1,0,0,0,0,0])
    if(label[i]==2):
        x3.append(data[i])
        l3.append([0,0,1,0,0,0,0])
    if(label[i]==3):
        x4.append(data[i])
        l4.append([0,0,0,1,0,0,0])
    if(label[i]==4):
        x5.append(data[i])
        l5.append([0,0,0,0,1,0,0])
    if(label[i]==5):
        x6.append(data[i])
        l6.append([0,0,0,0,0,1,0])
    if(label[i]==6):
        x7.append(data[i])
        l7.append([0,0,0,0,0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
x4=np.array(x4)
x5=np.array(x5)
x6=np.array(x6)
x7=np.array(x7)

l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)
l4=np.array(l4)
l5=np.array(l5)
l6=np.array(l6)
l7=np.array(l7)

print(l7.shape)

(209, 7)


In [14]:
from sklearn.utils import shuffle
data, label = shuffle(data, label, random_state = 0)

In [15]:
# 80% train 20% test (NDVIclean)
X_train = data[0:436,:]
X_test = data[436:,:]
y_train = label[0:436]
y_test = label[436:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(436, 13)
(110, 13)
(436,)
(110,)


In [16]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(436, 7)
(110, 7)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 21

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.58126652
Epoch: 2 cost = 0.50029623
Epoch: 3 cost = 0.43436351
Epoch: 4 cost = 0.38923663
Epoch: 5 cost = 0.36136616
Epoch: 6 cost = 0.34194658
Epoch: 7 cost = 0.32641575
Epoch: 8 cost = 0.31379225
Epoch: 9 cost = 0.30360334
Epoch: 10 cost = 0.29510053
Epoch: 11 cost = 0.28754702
Epoch: 12 cost = 0.28044902
Epoch: 13 cost = 0.27353628
Epoch: 14 cost = 0.26666361
Epoch: 15 cost = 0.25974252
Epoch: 16 cost = 0.25273409
Epoch: 17 cost = 0.24569250
Epoch: 18 cost = 0.23879000
Epoch: 19 cost = 0.23222482
Epoch: 20 cost = 0.22605715
Epoch: 21 cost = 0.22017047
Epoch: 22 cost = 0.21437829
Epoch: 23 cost = 0.20854377
Epoch: 24 cost = 0.20265978
Epoch: 25 cost = 0.19690477
Epoch: 26 cost = 0.19190829
Epoch: 27 cost = 0.19020254
Epoch: 28 cost = 0.19058386
Epoch: 29 cost = 0.17787776
Epoch: 30 cost = 0.17208684
Epoch: 31 cost = 0.16519891
Epoch: 32 cost = 0.16053238
Epoch: 33 cost = 0.15373749
Epoch: 34 cost = 0.14215578
Epoch: 35 cost = 0.14354319
Epoch: 36 cost = 0.16451126
E

Epoch: 291 cost = 0.00801348
Epoch: 292 cost = 0.00903247
Epoch: 293 cost = 0.00958334
Epoch: 294 cost = 0.00959740
Epoch: 295 cost = 0.00891644
Epoch: 296 cost = 0.00831998
Epoch: 297 cost = 0.00855022
Epoch: 298 cost = 0.00864285
Epoch: 299 cost = 0.00997346
Epoch: 300 cost = 0.27074670
Epoch: 301 cost = 0.17601613
Epoch: 302 cost = 0.07211401
Epoch: 303 cost = 0.06420625
Epoch: 304 cost = 0.03921216
Epoch: 305 cost = 0.01147110
Epoch: 306 cost = 0.00963470
Epoch: 307 cost = 0.00913581
Epoch: 308 cost = 0.00882350
Epoch: 309 cost = 0.00881572
Epoch: 310 cost = 0.00882316
Epoch: 311 cost = 0.00884243
Epoch: 312 cost = 0.00895649
Epoch: 313 cost = 0.00937641
Epoch: 314 cost = 0.00886617
Epoch: 315 cost = 0.00857898
Epoch: 316 cost = 0.00909450
Epoch: 317 cost = 0.00900850
Epoch: 318 cost = 0.00856711
Epoch: 319 cost = 0.00815956
Epoch: 320 cost = 0.00826614
Epoch: 321 cost = 0.00885410
Epoch: 322 cost = 0.00881815
Epoch: 323 cost = 0.00961854
Epoch: 324 cost = 0.00931499
Epoch: 325 cos

Epoch: 602 cost = 0.00511640
Epoch: 603 cost = 0.00494872
Epoch: 604 cost = 0.00469843
Epoch: 605 cost = 0.00512238
Epoch: 606 cost = 0.00542604
Epoch: 607 cost = 0.00537875
Epoch: 608 cost = 0.00507461
Epoch: 609 cost = 0.00494482
Epoch: 610 cost = 0.00526954
Epoch: 611 cost = 0.00536512
Epoch: 612 cost = 0.00577154
Epoch: 613 cost = 0.00573338
Epoch: 614 cost = 0.00587524
Epoch: 615 cost = 0.00536312
Epoch: 616 cost = 0.00492590
Epoch: 617 cost = 0.00487711
Epoch: 618 cost = 0.00528297
Epoch: 619 cost = 0.00507633
Epoch: 620 cost = 0.00485483
Epoch: 621 cost = 0.00549555
Epoch: 622 cost = 0.00661612
Epoch: 623 cost = 0.00539757
Epoch: 624 cost = 0.00504453
Epoch: 625 cost = 0.00577642
Epoch: 626 cost = 0.00554247
Epoch: 627 cost = 0.00660862
Epoch: 628 cost = 0.00731253
Epoch: 629 cost = 0.24446496
Epoch: 630 cost = 0.08509829
Epoch: 631 cost = 0.01998292
Epoch: 632 cost = 0.00671546
Epoch: 633 cost = 0.00624464
Epoch: 634 cost = 0.00548603
Epoch: 635 cost = 0.00503677
Epoch: 636 cos

Epoch: 916 cost = 0.00463908
Epoch: 917 cost = 0.00466974
Epoch: 918 cost = 0.00428119
Epoch: 919 cost = 0.00471886
Epoch: 920 cost = 0.00454103
Epoch: 921 cost = 0.00427683
Epoch: 922 cost = 0.00478625
Epoch: 923 cost = 0.00488995
Epoch: 924 cost = 0.00525629
Epoch: 925 cost = 0.00505751
Epoch: 926 cost = 0.00456645
Epoch: 927 cost = 0.00414645
Epoch: 928 cost = 0.00425562
Epoch: 929 cost = 0.00477266
Epoch: 930 cost = 0.00467341
Epoch: 931 cost = 0.00465790
Epoch: 932 cost = 0.00440593
Epoch: 933 cost = 0.00460887
Epoch: 934 cost = 0.00472370
Epoch: 935 cost = 0.00463704
Epoch: 936 cost = 0.00429469
Epoch: 937 cost = 0.00503333
Epoch: 938 cost = 0.00494795
Epoch: 939 cost = 0.00568997
Epoch: 940 cost = 0.00487899
Epoch: 941 cost = 0.00482330
Epoch: 942 cost = 0.00468700
Epoch: 943 cost = 0.00456935
Epoch: 944 cost = 0.00426988
Epoch: 945 cost = 0.00416987
Epoch: 946 cost = 0.00438988
Epoch: 947 cost = 0.00490659
Epoch: 948 cost = 0.00468843
Epoch: 949 cost = 0.00506985
Epoch: 950 cos

In [18]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
print(y_test.shape)

(110, 7)


In [21]:
# on 1000 epochcura
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.79999995


In [31]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
res4=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x4, Y: l4}))
res5=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x5, Y: l5}))
res6=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x6, Y: l6}))
res7=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x7, Y: l7}))
print(res6)

[5 5 5 5 1 1 1 5 5 1 1 2 5 5 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5]


In [32]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
x4=np.sum(np.equal(res1, 3))
x5=np.sum(np.equal(res1, 4))
x6=np.sum(np.equal(res1, 5))
x7=np.sum(np.equal(res1, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
x4=np.sum(np.equal(res2, 3))
x5=np.sum(np.equal(res2, 4))
x6=np.sum(np.equal(res2, 5))
x7=np.sum(np.equal(res2, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
x4=np.sum(np.equal(res3, 3))
x5=np.sum(np.equal(res3, 4))
x6=np.sum(np.equal(res3, 5))
x7=np.sum(np.equal(res3, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res4, 0))
x2=np.sum(np.equal(res4, 1))
x3=np.sum(np.equal(res4, 2))
x4=np.sum(np.equal(res4, 3))
x5=np.sum(np.equal(res4, 4))
x6=np.sum(np.equal(res4, 5))
x7=np.sum(np.equal(res4, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res5, 0))
x2=np.sum(np.equal(res5, 1))
x3=np.sum(np.equal(res5, 2))
x4=np.sum(np.equal(res5, 3))
x5=np.sum(np.equal(res5, 4))
x6=np.sum(np.equal(res5, 5))
x7=np.sum(np.equal(res5, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res6, 0))
x2=np.sum(np.equal(res6, 1))
x3=np.sum(np.equal(res6, 2))
x4=np.sum(np.equal(res6, 3))
x5=np.sum(np.equal(res6, 4))
x6=np.sum(np.equal(res6, 5))
x7=np.sum(np.equal(res6, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res7, 0))
x2=np.sum(np.equal(res7, 1))
x3=np.sum(np.equal(res7, 2))
x4=np.sum(np.equal(res7, 3))
x5=np.sum(np.equal(res7, 4))
x6=np.sum(np.equal(res7, 5))
x7=np.sum(np.equal(res7, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

confusion=np.array(confusion)
print(confusion)

[[ 48   9   4   0   0   0   0]
 [ 12 146   2   0   2   9   1]
 [ 16   3   8   0   4   0   2]
 [  1   8   1   0   0   0   0]
 [  5   3   5   0  12   0   2]
 [  0   6   1   0   0  27   0]
 [  5   7   0   0   2   1 194]]


In [25]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data[0:382,:]
X_test = data[382:,:]
y_train = label[0:382]
y_test = label[382:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(382, 13)
(164, 13)
(382,)
(164,)


In [26]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(382, 7)
(164, 7)


In [27]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 19

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.60159347
Epoch: 2 cost = 0.57447233
Epoch: 3 cost = 0.55056359
Epoch: 4 cost = 0.52934401
Epoch: 5 cost = 0.51051845
Epoch: 6 cost = 0.49381617
Epoch: 7 cost = 0.47884419
Epoch: 8 cost = 0.46506187
Epoch: 9 cost = 0.45186372
Epoch: 10 cost = 0.43870741
Epoch: 11 cost = 0.42522482
Epoch: 12 cost = 0.41129903
Epoch: 13 cost = 0.39711849
Epoch: 14 cost = 0.38320234
Epoch: 15 cost = 0.37031800
Epoch: 16 cost = 0.35917615
Epoch: 17 cost = 0.34999120
Epoch: 18 cost = 0.34233556
Epoch: 19 cost = 0.33550280
Epoch: 20 cost = 0.32895265
Epoch: 21 cost = 0.32243867
Epoch: 22 cost = 0.31591356
Epoch: 23 cost = 0.30942055
Epoch: 24 cost = 0.30303052
Epoch: 25 cost = 0.29680913
Epoch: 26 cost = 0.29079843
Epoch: 27 cost = 0.28501038
Epoch: 28 cost = 0.27943031
Epoch: 29 cost = 0.27402708
Epoch: 30 cost = 0.26876516
Epoch: 31 cost = 0.26361442
Epoch: 32 cost = 0.25855532
Epoch: 33 cost = 0.25357958
Epoch: 34 cost = 0.24868704
Epoch: 35 cost = 0.24388120
Epoch: 36 cost = 0.23916590
E

Epoch: 313 cost = 0.00033571
Epoch: 314 cost = 0.00056434
Epoch: 315 cost = 0.00078378
Epoch: 316 cost = 0.00057438
Epoch: 317 cost = 0.00036178
Epoch: 318 cost = 0.00035498
Epoch: 319 cost = 0.00042814
Epoch: 320 cost = 0.00067663
Epoch: 321 cost = 0.00063497
Epoch: 322 cost = 0.00046832
Epoch: 323 cost = 0.00054059
Epoch: 324 cost = 0.00048815
Epoch: 325 cost = 0.00036614
Epoch: 326 cost = 0.00042657
Epoch: 327 cost = 0.00060456
Epoch: 328 cost = 0.00053704
Epoch: 329 cost = 0.00057331
Epoch: 330 cost = 0.00048244
Epoch: 331 cost = 0.00038026
Epoch: 332 cost = 0.00033720
Epoch: 333 cost = 0.00038260
Epoch: 334 cost = 0.00046152
Epoch: 335 cost = 0.00059746
Epoch: 336 cost = 0.00040501
Epoch: 337 cost = 0.00041933
Epoch: 338 cost = 0.00066879
Epoch: 339 cost = 0.00050117
Epoch: 340 cost = 0.00038439
Epoch: 341 cost = 0.00047192
Epoch: 342 cost = 0.00049385
Epoch: 343 cost = 0.00042531
Epoch: 344 cost = 0.00036474
Epoch: 345 cost = 0.00028263
Epoch: 346 cost = 0.00057071
Epoch: 347 cos

Epoch: 656 cost = 0.00032693
Epoch: 657 cost = 0.00019891
Epoch: 658 cost = 0.00021375
Epoch: 659 cost = 0.00030688
Epoch: 660 cost = 0.00022108
Epoch: 661 cost = 0.00022350
Epoch: 662 cost = 0.00022621
Epoch: 663 cost = 0.00027904
Epoch: 664 cost = 0.00016343
Epoch: 665 cost = 0.00016522
Epoch: 666 cost = 0.00033881
Epoch: 667 cost = 0.00036080
Epoch: 668 cost = 0.00029584
Epoch: 669 cost = 0.00018251
Epoch: 670 cost = 0.00022969
Epoch: 671 cost = 0.00018586
Epoch: 672 cost = 0.00021258
Epoch: 673 cost = 0.00026160
Epoch: 674 cost = 0.00017682
Epoch: 675 cost = 0.00022731
Epoch: 676 cost = 0.00035329
Epoch: 677 cost = 0.00025594
Epoch: 678 cost = 0.00030578
Epoch: 679 cost = 0.00025053
Epoch: 680 cost = 0.00012276
Epoch: 681 cost = 0.00018305
Epoch: 682 cost = 0.00019906
Epoch: 683 cost = 0.00020142
Epoch: 684 cost = 0.00022526
Epoch: 685 cost = 0.00019474
Epoch: 686 cost = 0.00033495
Epoch: 687 cost = 0.00047728
Epoch: 688 cost = 0.00035282
Epoch: 689 cost = 0.00025188
Epoch: 690 cos

Epoch: 971 cost = 0.00026623
Epoch: 972 cost = 0.00018469
Epoch: 973 cost = 0.00011000
Epoch: 974 cost = 0.00021415
Epoch: 975 cost = 0.00017085
Epoch: 976 cost = 0.00013089
Epoch: 977 cost = 0.00011242
Epoch: 978 cost = 0.00013991
Epoch: 979 cost = 0.00018848
Epoch: 980 cost = 0.00026594
Epoch: 981 cost = 0.00021824
Epoch: 982 cost = 0.00013506
Epoch: 983 cost = 0.00026256
Epoch: 984 cost = 0.00023924
Epoch: 985 cost = 0.00020663
Epoch: 986 cost = 0.00018666
Epoch: 987 cost = 0.00017417
Epoch: 988 cost = 0.00015050
Epoch: 989 cost = 0.00009844
Epoch: 990 cost = 0.00010868
Epoch: 991 cost = 0.00032333
Epoch: 992 cost = 0.00028027
Epoch: 993 cost = 0.00017695
Epoch: 994 cost = 0.00024956
Epoch: 995 cost = 0.00016215
Epoch: 996 cost = 0.00019518
Epoch: 997 cost = 0.00014491
Epoch: 998 cost = 0.00007395
Epoch: 999 cost = 0.00012626
Epoch: 1000 cost = 0.00021905


In [28]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.76829267
